Hello

In [ ]:
import pandas as pd

In [1]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pandas as pd


# Weighted Łukasiewicz Logic
class WeightedLukasiewiczLogic:
    def conjunction(self, inputs, weights, beta=1.0):
        """
        Weighted Łukasiewicz Conjunction (AND).
        """
        return torch.clamp(beta - torch.sum(weights * (1 - inputs), dim=-1), min=0, max=1)

    def disjunction(self, inputs, weights, beta=1.0):
        """
        Weighted Łukasiewicz Disjunction (OR).
        """
        return torch.clamp(1 - beta + torch.sum(weights * inputs, dim=-1), min=0, max=1)

    def implication(self, p, q, w_p=1.0, w_q=1.0, beta=1.0):
        """
        Weighted Łukasiewicz Implication (IF-THEN).
        """
        return torch.clamp(1 - beta + w_p * (1 - p) + w_q * q, min=0, max=1)


# Logical Neuron
class LogicalNeuron:
    def __init__(self, logic):
        self.logic = logic

    def compute_bounds(self, inputs, operation, weights=None, beta=1.0):
        if operation == "AND":
            return self.logic.conjunction(inputs, weights, beta)
        elif operation == "OR":
            return self.logic.disjunction(inputs, weights, beta)
        elif operation == "IMPLICATION":
            p, q = inputs[:, 0], inputs[:, 1]
            return self.logic.implication(p, q, weights[0], weights[1], beta)
        else:
            raise ValueError("Unsupported operation")


# Dataset Class
class LogicalReasoningDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len, is_test=False):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.is_test = is_test

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        context = row['context']
        question = row['question']
        text = f"{context} [SEP] {question}"

        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        item = {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
        }

        if not self.is_test:
            item['label'] = torch.tensor(row['label'], dtype=torch.long)

        return item


# Logical Neural Network Model
class LogicalNeuralNetwork(nn.Module):
    def __init__(self, base_model, num_labels, logic, lambda_weight=0.5):
        super(LogicalNeuralNetwork, self).__init__()
        self.base_model = base_model
        self.num_labels = num_labels
        self.logic_layer = LogicalNeuron(logic)
        self.lambda_weight = lambda_weight

    def forward(self, input_ids, attention_mask, labels=None, constraints=None):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=-1)  # Convert logits to probabilities
        loss = None

        if labels is not None:
            # Standard classification loss
            ce_loss = nn.CrossEntropyLoss()(logits, labels)

            # Logical constraints loss
            logic_loss = 0
            if constraints:
                for constraint in constraints:
                    operation = constraint['operation']
                    inputs = torch.stack([probabilities[:, i] for i in constraint['inputs']], dim=-1)
                    weights = constraint.get('weights', [1.0] * len(constraint['inputs']))
                    beta = constraint.get('beta', 1.0)

                    logic_out = self.logic_layer.compute_bounds(inputs, operation, weights, beta)
                    logic_loss += torch.mean(1 - logic_out)  # Penalize logical violations

            loss = ce_loss + self.lambda_weight * logic_loss

        return (loss, logits)


# Training Function
def train_model(model, train_dataloader, optimizer, device, epochs=3):
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        print(f"Epoch {epoch + 1}/{epochs}")

        for batch in tqdm(train_dataloader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            # Example constraints
            constraints = [
                {"operation": "IMPLICATION", "inputs": [0, 1], "weights": [1.0, 1.0], "beta": 1.0},
                {"operation": "AND", "inputs": [1, 2], "weights": [1.0, 1.0], "beta": 1.0},
            ]

            optimizer.zero_grad()
            loss, _ = model(input_ids, attention_mask, labels=labels, constraints=constraints)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch + 1} Loss: {total_loss / len(train_dataloader):.4f}")


# Prediction Function
def generate_predictions(model, test_dataloader, device):
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in tqdm(test_dataloader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            _, logits = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(logits, dim=1)
            predictions.extend(preds.cpu().numpy())

    return predictions


# Main Function
def main():
    train_file = "train.csv"
    test_file = "test.csv"
    sample_submission_file = "sample_submission.csv"

    # Load data
    train_data = pd.read_csv(train_file)
    test_data = pd.read_csv(test_file)

    # Split train_data into training and validation sets
    train_df, val_df = train_test_split(train_data, test_size=0.2, random_state=42)

    # Initialize tokenizer and model
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    base_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)
    logic = WeightedLukasiewiczLogic()
    model = LogicalNeuralNetwork(base_model, num_labels=4, logic=logic, lambda_weight=0.5)

    # Create datasets and dataloaders
    train_dataset = LogicalReasoningDataset(train_df, tokenizer, max_len=512)
    test_dataset = LogicalReasoningDataset(test_data, tokenizer, max_len=512, is_test=True)

    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

    # Device setup
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

    # Train the model
    train_model(model, train_dataloader, optimizer, device, epochs=3)

    # Generate predictions
    predictions = generate_predictions(model, test_dataloader, device)

    # Save predictions
    sample_submission = pd.read_csv(sample_submission_file)
    sample_submission['label'] = predictions
    sample_submission.to_csv('submission.csv', index=False)
    print("Predictions saved to submission.csv")





In [ ]:
if __name__ == "__main__":
    main()